---
# __Machine Learning Workshop Part-B__

## ___Learning objectives___

At the end of the exercise, you should be able to:
- Conduct an end-to-end classification analysis given data
- Interpret model with example global and local interpretaion methods.

In [ ]:
# First thing first: test your install
import sklearn, pandas, matplotlib, seaborn, imblearn, numpy, shap, tqdm

# If you encounter error, talk to the instructors and/or your neighbor ASAP

## ___Outline___

- Session 1
  - Intro to machine learning
  - [Part A review](#review)
  - [Step 1. Define ML problem](#step1)
  - [Step 2. Exploratory data analysis](#step2)
  - [Step 3: Split train/test](#step3)
- Session 2
  - [Step 4: Feature engineering](#step4)
  - [Step 5: Select model](#step5)
  - [Step 6: Repeat 2-5](#step6)
- Session 3
  - [Step 7: Evalute model](#step7)
  - [Step 8: Interpret model](#step8)
  - Workshop wrap-up

----
<a id="review"></a>

## ___Part A review___

&#9989; <font color=red>**QUESTION:**</font> In __Part A__, we have gone thorugh the major steps. __In the next 5 minutes__, discuss with your neighbors: What is your understanding of each step? Is there any step or term that is confusng to you? 

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

----
<a id="step1"></a>
## ___Step 1. Define ML problem___

___Problem statement___: 
- Given:
  - Known genes in general metabolism (GM) or specialized metabolism (SM)
  - Various features of genes
    - E.g., expression levels, functional category they belong to 
- How can we use them to:
  - Distinguish genes involved in GM from those involved in SM?


&#9989; <font color=red>**QUESTION:**</font> __In the next two minutes__, discuss with your neighbors: what features should we use to best distinguish GM and SM genes?

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

----
<a name="step2"></a>
## __Step 2. Exploratory data analysis (EDA)__

&#9989; **<font color=blue>DO THIS:</font>** Run the following cell to load the data. Note that we set a random seed (`rand_seed`). This is so we can reprdouce the random data generated along the way so others can repeat the same analysis and get the similar, if not the same results.

In [ ]:
import pandas as pd

rand_seed = 20240507

# Load the dataset from the file enzyme_gene.csv as a Pandas DataFrame where the
# `Gene` column is read in as index.
enzyme_gene = pd.read_csv("enzyme_gene.csv", index_col=0)

# get all feature names
features = enzyme_gene.columns[1:] 

# get all feature names
# Print out 4 randomly sampled instances
enzyme_gene.sample(4)

&#9989; **<font color=red>QUESTION:</font>** __In the next minute__, discuss with your neighbors: which features do you think will be the best for distinguishing GM and SM genes? For what the feature names mean, see [this spreadsheet](https://docs.google.com/spreadsheets/d/1VjnlJgKsGC8GNNds7VtbYfjAQpjETkZNI6AlF9c2DAY/edit?usp=sharing). Also, put your name down for the feature you think will be the best!

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

### ___2.1 Univariate EDA___

&#9989; **<font color=blue>DO THIS:</font>** Earlier, we use `enzyme_gene.sample(4)` to get a few samples from the `enzyme_gene` dataframe. Replace `sample(4)` with `describe()` and see what information become available.

In [ ]:
# Put your code here
# If you are not sure how to proceed, look at the answer in the following cell.


In [ ]:
##ANSWER##
# Generate summary statics for each features
enzyme_gene.describe()
##ANSWER##

&#9989; **<font color=blue>DO THIS:</font>** Tell us more about `enzyme_gene` by using `shape` and `nunique()`:

In [ ]:
# put your codes here

# Display the # of columns and rows
print("\nShape:", enzyme_gene.shape)

# Display the # of unique values in each feature
print("\n### Number unique:\n", enzyme_gene.nunique())

&#9989; **<font color=blue>DO THIS:</font>** Run the following to determine how many entires have `GM`, `SM`, and `Unknown` labels, respectively.

In [ ]:
print(enzyme_gene["Label"].value_counts())

&#9989; **<font color=blue>DO THIS:</font>** Determine the number of `null` entries in each column and print them out.

In [ ]:
enzyme_gene.isnull().sum()

### ___2.2 Univariate graphical EDA___

&#9989; **<font color=blue>DO THIS:</font>** Plot the histograms of all features.

In [ ]:
# The 1st ":" is to get all rows. The "1:" part is to get the 2nd column and on.
feature_values = enzyme_gene.iloc[:, 1:]

# Draw histogram
hist = feature_values.hist(figsize=(12,12), bins=50)


&#9989; **<font color=red>QUESTION:</font>**  __In the next 1 minute__, discuss with your neighbors: did you see any issues with the dataset based on the above analyses?

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

### ___2.3 Multi-variate non-graphical EDA___

&#9989; **<font color=blue>DO THIS:</font>** Determine pairwise Spearman's rank correlations of all features:

In [ ]:
# Calculate Spearman's rank correlations for all feature pairs
corr = feature_values.corr(method = 'spearman')
corr

### ___2.4 Multi-variate graphical EDA___

&#9989; **<font color=blue>DO THIS:</font>** Plot the pairwise Spearman's rank correlations of all features as a heatmap using Seaborn.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(7,6))
sns.heatmap(corr, cmap="coolwarm")

&#9989; **<font color=red>QUESTION:</font>**  __In the next 1 minute__, discuss with your neighbors: did you see any issues with the dataset based on the univariate analyses?

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

---
<a name="step3"></a>
## __Step 3: Split train/test__

The best practice is alway to set aside testing data __as the FIRST STEP__ or as early as possible. This way, the testing data is truly independent from any of the modeling process aside from the processing steps needed. Nonetheless, there may be things you need to take care of first before splitting the data. In this example, we need to rid of unwanted instances before data split.


### ___3.1 Deal with unwanted instances___

&#9989; **<font color=blue>DO THIS:</font>** Filter data so only instances with `SM` and `GM` labels are kept.

In [ ]:
labels              = ['GM', 'SM']
label_column        = enzyme_gene['Label']
label_column_filter = label_column.isin(labels)

# enzyme_gene dataframe with only GM and SM
enzyme_gene_fil = enzyme_gene[label_column_filter]

# Count the occurence of unique values
enzyme_gene_fil['Label'].value_counts()

&#9989; **<font color=blue>DO THIS:</font>** For classification tasks, class values are typically integers instead of texts (like SM or GM here). So, we will convert `GM` and `SM` to 0 and 1, respectively. 

Write code at the end to show that the filtering is working.

In [ ]:
# import the proprecessing functions
from sklearn import preprocessing

# Create a LabelEncoder object: this is simply a software tool that turn 
# (encode) texts into 0 or 1 (labels) in this case.
le = preprocessing.LabelEncoder()

# Send the Label column of enzyme_gene_fil dataframe to the LabelEncoder so
# it can fit (i.e., learn) how to encode the labels.
le.fit(enzyme_gene_fil.Label)

# Now, used the fitted (learned) encoder to transform texts to labels
enzyme_gene_fil['Label'] = le.transform(enzyme_gene_fil.Label)

# Write code below to show that the Label encoding is working: 0s and 1s


# If you are not sure how to proceed, look at the answer in the following cell.

In [ ]:
##ANSWER##
enzyme_gene_fil['Label'].value_counts()
##ANSWER##

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

### ___3.2 Split training/testing sets___

&#9989; **<font color=blue>DO THIS:</font>** Let's split the training and testing data. 

Please comments on the lines as indicated.

In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset into training and testing sets
train, test = train_test_split(
                enzyme_gene_fil,                # The data to split
                test_size=0.2,                  # Proportion data for testing
                stratify=enzyme_gene_fil.Label, # Make sure proportions of 0/1
                                                # labels are similar between
                                                # training and testing sets
                random_state=rand_seed)

# Print out proportions of different labels in the training data
print(train['Label'].value_counts()/train.shape[0])

# Print out proportions of different labels in the testing data
print(test['Label'].value_counts()/test.shape[0])

&#9989; **<font color=red>QUESTION:</font>**  In __the next 2 minutes__, discuss with your neighbor: what's the point of splitting training and testing data again? How big should the testing data be?

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

---
<a name="step4"></a>
## __Step 4: Feature engineering__

Feature engineering involves processing, transforming, selecting, combining features in ways that will improve the model. 

### ___4.1 Deal with missing data___

We will just try to deal with this in one way. __In reality__, You need to try multiple approaches to see how you can get the best results.

&#9989; **<font color=blue>DO THIS:</font>** First let's remind ourself how many instances are there after we get rid of `unknown`.

In [ ]:
enzyme_gene_fil.shape

&#9989; **<font color=blue>DO THIS:</font>** Let's drop any rows with >25% missing values and see how many instances are still there.

In [ ]:
# ask which values are null.
row_na     = enzyme_gene_fil.isnull()
row_na

In [ ]:
# count the mising values (null, NA, or called NaN: Not a Number) of each crow
row_na_num = train.isnull().sum(axis=1)
row_na_num[:5]

In [ ]:
num_feat     = train.shape[1] - 1            # number of features in the data
rows_to_keep = row_na_num/num_feat < 0.25    # rows with <25% missing values
train_keep = train[rows_to_keep]             # training data with rows to keep 
train_keep['Label'].value_counts()

&#9989; **<font color=blue>DO THIS:</font>** A lot of data is removed but this is much better than just drop any row with missing values. Next, let's try to impute the missing values with `KNNImputer`.

In [ ]:
from sklearn.impute import KNNImputer

# Create an imputer object to imptue our data
# n_neighbors is the number of neighbors used to estimate the missing values.
imputer = KNNImputer(n_neighbors=5)

# Train the imputer with training data
imputer.fit(train_keep)

# Transform missing values into imputed values, hence train_keep_imp (imputed)
train_keep_imp = imputer.transform(train_keep)

# The thing with KNNImputer is it create train_keep_imp as a Numpy array so
# we don't have the column names any more. Because I really want to know what 
# these columns are, so let's turn this back to a DataFrame with column names.
train_keep_imp = pd.DataFrame(train_keep_imp, columns=train.columns)
train_keep_imp.sample(4)

&#9989; **<font color=orange>CAUTION:</font>** There is a hyperparamter here: `n_neighbors`. We set it to 5 here but __in reality__, multiple values need to be evaluated. Also, you DO NOT impute labels. We did not exclude the label column because we have make sure there is no missing value early on. So imputation will not impact it.

&#9989; **<font color=blue>DO THIS:</font>** Follow what we have done for the training data. Write code that will impute the testing set:
1. Drop any rows with > 25% missing values.
2. Impute missing value with KNNImputer.
3. Check that there is no missing value.

In [ ]:
# put your code here


# I encourage you to try to figure this out. If you get stuck. Look at the 
# answer below and comments on what each line does.

In [ ]:
##ANSWER##
# Sum the number of NAs in each row of the testing set
row_na_num = test.isnull().sum(axis=1)

# Get the number of features
num_feat   = test.shape[1] - 1

# Determine which rows have below threshold (25%) NAs
row_na_below_threshold = row_na_num/num_feat < 0.25

# Get the rows we want to keep
test_keep  = test[row_na_below_threshold]

# Impute missing values
test_keep_imp = imputer.transform(test_keep)
test_keep_imp = pd.DataFrame(test_keep_imp, columns=test.columns)
test_keep_imp.isnull().sum()
##ANSWER##

&#9989; **<font color=red>QUESTION:</font>**  In __the next 2 minutes__, discuss with your neighbor: note that here `testing` data is not used to `fit` the imputer. Instead, the imputer fitted with training data is used to `tranform` test set. Why is that?

### &#9978; **<font color=purple>PAUSE: when you finish, please turn your attention to the instructor. </font>**

### ___4.2 Deal with data imbalance___

&#9989; **<font color=blue>DO THIS:</font>** Here we will use a hybrid approach:
- Up-sample the minority class so it has twice as many instances __AND__ 
- Downsample the majority class so it is the same number as the minority.

In [ ]:
train_keep_imp.head()

In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# training features
X_train = train_keep_imp.iloc[:,1:] 

# training labels
y_train = train_keep_imp.iloc[:,0]  

# This will be used in many other occasions.
feat_names = X_train.columns 

# summarize class distribution
counter = Counter(y_train)
print("Before:", counter)

# Over-sample minority, under-sample majority
over = SMOTE(sampling_strategy=0.4)
under = RandomUnderSampler(sampling_strategy=1)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

# transform the dataset
X_train_bal, y_train = pipeline.fit_resample(X_train, y_train)

# summarize the new class distribution
counter = Counter(y_train)
print("After :", counter)

&#9989; **<font color=red>QUESTION:</font>** In __the next 2 minutes__, discuss with your neighbor: Resampling, particularly upsampling can __only__ be applied to the training data. The testing set __should not__ be changed in this step. Why is that?

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

### ___4.3. Deal with data scaling___

&#9989; **<font color=blue>DO THIS:</font>** Bassed on your exploratory data analsysi you probably the data range differ widely. Before we work on scaling the data, let's see how the data ranges differ:

In [ ]:
X_train_bal.describe()

&#9989; **<font color=blue>DO THIS:</font>**  In the cell below, let's use `RobustScaler` to scale the balanced training data feature values (`X_train_bal`).

__DO NOT__ applying scaling to the labels (`y`).

Call the scaled features as `X_train_scale`.

In [ ]:
from sklearn.preprocessing import RobustScaler

# initialize a scaler
scaler = RobustScaler()

# fit the scaler with training features
scaler.fit(X_train_bal)

# transform the training feature values with the fitted scaler
X_train_scale = scaler.transform(X_train_bal)
X_train_scale = pd.DataFrame(X_train_scale, columns=X_train.columns)
X_train_scale.describe()

&#9989; **<font color=blue>DO THIS:</font>** Provide code below to process testing data so it is scaled the same way.
- Create `X_test` and `y_test` with `test_keep_imp`.
- Transform (but __do not__ fit) `X_test` with the `RobustScaler`.

In [ ]:
# put your code here


# If you don't feel comfortable doing this, check out the answer below and
# comment on what they are doing.

In [ ]:
# put your code here

##ANSWER##
X_test = test_keep_imp.iloc[:,1:]
y_test = test_keep_imp.iloc[:,0]

X_test_scale = scaler.transform(X_test)
X_test_scale = pd.DataFrame(X_test_scale, columns=X_test.columns)
##ANSWER##

&#9989; **<font color=orange>CAUTION:</font>** We __did not__ deal with non-normal distributions or co-linear features. In practice, they need to be dealth with. To make the exercise simpler, we did not include categorical variable. For starter, please checkout, after class:
- [This post on data transformation](https://www.analyticsvidhya.com/blog/2020/07/types-of-feature-transformation-and-scaling/).
- [This post on colinearity](https://towardsdatascience.com/multicollinearity-in-data-science-c5f6c0fe6edf).
- [This post on how to work with categorical variables](https://machinelearningmastery.com/one-hot-encoding-for-categorical-data/).
- More generally, [here is a good article on feature engineering](https://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/).

### &#9978; **<font color=purple>PAUSE: please turn your attention to the instructor. </font>**

---
<a name="step5"></a>
## __Step 5: Select model__

### ___5.1 Random forest___

&#9989; **<font color=blue>DO THIS:</font>** Here we will not go into details on how the RandomForest algorithm works. there is a substantial number of good tutorial/blog posts on RandomForest (e.g., [this one](https://machinelearningmastery.com/bagging-and-random-forest-ensemble-algorithms-for-machine-learning/)) and I encourage you to look into it. Comments on the major steps as indicated.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# Create a function for running RandomForest
def run_randomforest(X_train, y_train):
    # Below is a Python dictionary specify the hyperparameters to be tested
    #  2x3x4x1 = 24
    param_grid = {'n_estimators': [200, 500],
                  'max_features': ['auto', 'sqrt', 'log2'],
                  'max_depth' : [3,5,7,9],
                  'criterion' :['entropy']}

    # Initialize a random forest classifier (rfc) with a random seed
    rfc = RandomForestClassifier(random_state=rand_seed)

    # Initialize a grid search object that will search through each of the 24
    # hyperparameter combinations. For each combination, a five fold cross-
    # validation (cv) is done. So totally 24x5 = 120 random forest classifiers 
    # will be build.
    rfc_gs = GridSearchCV(
                rfc,
                param_grid,
                cv=5,              # cross validation folds
                verbose=2,         # 
                scoring='roc_auc', # find model with the best ROC-AUC
                n_jobs=8)          # number of concurrent jobs, you need to
                                   # adjust this based on the number of CPU cores
                                   # available on your machine.

    # Pass the training feaure and label data to the grid search object and
    # start fitting (training) models
    rfc_gs.fit(X_train, y_train)

    # Return the fitted grid search object
    return rfc_gs

In [ ]:
# Call the run_randomforest function defined above
rfc_gs = run_randomforest(X_train_scale, y_train)

&#9989; **<font color=blue>DO THIS:</font>** In the `grid_search` object, there is whole punch of useful information. Run the following cells.

In [ ]:
# The best model (also called estimator)
best_model = rfc_gs.best_estimator_

# Note that the best hyperparameters are also reported
best_model

In [ ]:
# The best ROC-AUC score averged across CV folds for the best model
print(rfc_gs.best_score_)

&#9989; **<font color=blue>DO THIS:</font>** Although we finished the run rather quickly here, a typical model fitting process can take hours or even days! Thus, the models should be saved so you can reused them in the future. Run the following to save the best estimator. 

In [ ]:
import pickle

filename = "model_randomforest_gridsearch.save"

pickle.dump(rfc_gs.best_estimator_, open(filename, 'wb'))

&#9989; **<font color=red>QUESTION:</font>** In __the next 2 min__, discuss with your neighbors: what just happened here? Can you describe what you have accomplished in this step and what are done? Discuss with your neighbors.

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

### ___5.2 Support Vector Classifier (SVC)___

&#9989; **<font color=blue>DO THIS:</font>** There are [many other supervised learning algorithms in Scikit-Learn](https://scikit-learn.org/stable/supervised_learning.html). Let's use [Support Vector Machine](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC). 

Provide comment on the indicated lines.

In [ ]:
# Train a SVM classification model
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# COMMENT: What does this do?
#
param_grid = {'C': [1, 10, 1e2],
              'gamma': [0.0001, 0.001, 0.01, 0.1], 
              'kernel': ['linear', 'rbf']}

# COMMENT: What does this do?
# 
svc    = SVC()

# COMMENT: What does this do?
# 
svc_gs = GridSearchCV(svc, param_grid, cv=5, verbose=2, scoring='roc_auc',
                      n_jobs=8)

# COMMENT: What does this do?
# 
svc_gs.fit(X_train_scale, y_train)

# COMMENT: What does this do?
# 
filename = "model_svc_gridsearch.save"
pickle.dump(svc_gs.best_estimator_, open(filename, 'wb'))

# COMMENT: What does these do?
# 
print(svc_gs.best_params_)
print(svc_gs.best_score_)

##COMMENT ANSWERS##
#  Set up the hyperparameter combinations: 3x4x2 = 24 runs
#  Intialize a support vector classifier
#  Initiate a grid search object with cross validation
#  Search for the best hyperparameters with training data
#  Save the best model as a file
#  Print out the best parameters and best scores

&#9989; **<font color=orange>CAUTION:</font>** We just try two algorithms here, you should try a lot more.

In addition, beyond the hyperparameters associated with the algorithms, there are other things to tuned here:
- Cross validation methods: There are quite a number of approaches. See [this](https://scikit-learn.org/stable/modules/cross_validation.html) for examples.
- Searching parameters: Grid search is but one approach. Two other popular methods are [randomized search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) and [Bayesian optimization](https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html). These should also be tested.

&#9989; **<font color=red>QUESTION:</font>** In __the next 2 min__, discuss with your neighbors: based on the above two runs, can you decide which algorithm is better? Why and why not?

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

---
<a name="step6"></a>
## __Step 6. Repeat Step 2-5__

In a typical ML project, after you have explored different algorithms to find the best __initial__ model, it is time to go back to tweak everything to see if you can do even better. Things don't just end here!

An important step here is __feature selection__, a part of feature engineering, that involves selecting the most important features to rebuild your models.

### ___6.1 Get feature importance using trained model___

&#9989; **<font color=blue>DO THIS:</font>**  Uses the feature importance scores generated by the Random Forest model to choose the top features.


In [ ]:
from sklearn.inspection import permutation_importance

# Specify the best model (estimator) from our RandomForest run
rfc = rfc_gs.best_estimator_

# Calculate permutation importance of each feature
result = permutation_importance(
    rfc, X_train_scale, y_train, n_repeats=10, random_state=42, n_jobs=8)

# COMMENTS: What can you find in the result?
#
#
result

In [ ]:
##COMMENT ANSWER##
# importance_mean: mean importance value of each feature
# importance_std: imporrtance standard deviation of each feature
# importance: the importance scores of each feature for each repeat 

In [ ]:
# sort the permutation importance based on mean values
sorted_idx = result.importances_mean.argsort()[::-1]
sorted_idx

In [ ]:
# Get the importance values in order of the sorted_idx
importance_values = result.importances[sorted_idx].T

In [ ]:
# Get the feature names based on the sorted index
ordered_feature_label = X_train_scale.columns[sorted_idx]
ordered_feature_label

In [ ]:
# Plot the permutation importance results
fig, ax = plt.subplots(figsize=(6,6))
ax.boxplot(importance_values, 
           vert=False, 
           labels=ordered_feature_label)
ax.set_title("Permutation Importances (training set)")
fig.tight_layout()
plt.show()

&#9989; **<font color=red>QUESTION:</font>** In __the next 2 min__, discuss with your neighbors: how would you interpret the figure above? Is this as what you have expected when you hypothesize the most important feature?

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

### ___6.2 Retrain model using top 10 features___

&#9989; **<font color=blue>DO THIS:</font>**  A simpler model is always better, because it is easier to understand and because it tends not to be overfitted. Let's use the top 10 features to train a new RandomForest model and see how well it does.


In [ ]:
# Get top 10 feature names
feat_top10 = ordered_feature_label[:10]

# Get training data with only the top 10 features
X_train_top10 = X_train_scale[feat_top10]
X_train_top10.shape

In [ ]:
# Do the same for testing data
X_test_top10 = X_test_scale[feat_top10]
X_test_top10.shape

In [ ]:
# Train RandomForest model via grid search
rfc_gs_top10 = run_randomforest(X_train_top10, y_train)

In [ ]:
# Save model
filename = "model_randomforest_gridsearch_top10feat.save"

pickle.dump(rfc_gs_top10.best_estimator_, open(filename, 'wb'))

In [ ]:
# Get model score
rfc_gs_top10.best_score_

&#9989; **<font color=red>QUESTION:</font>** In __the next 2 min__, discuss with your neighbors: compare this result to the model with all features, should we just use the top 10? Why and why not?

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

---
<a name="step7"></a>
## __Step 7. Evaluate model with the testing set__

Assume that we are done with model building and have a final model that we cannot improve further. Then it is time to use the testing set to evaluate the model.

### ___7.1 Using the optimal model to predict testing set___

&#9989; **<font color=blue>DO THIS:</font>** Random Forest is a little better than SVC. And because using 10 features leads to a model that perofrm almost as well as the one with more features, we will pick the "optimal" model to be the random forest model using only 10 features.

In [ ]:
filename2 = "model_randomforest_gridsearch_top10feat.save"

# load model from file
rfc_loaded_top10 = pickle.load(open(filename2, 'rb')) # model using top 10

# predict testing data labels with the model using top 10 features
y_test_pred = rfc_loaded_top10.predict(X_test_top10)

In [ ]:
# Take a look at the 1st 40 predictions
print(y_test_pred[:40])

In [ ]:
# Also take a look at the 1st 40 TRUE values
print(numpy.array(y_test[:40]))

&#9989; <font color=red>**QUESTION:**</font> Based on the above results? Do you feel that the model is doing well?

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

### ___7.2 Confusion matrix___

&#9989; **<font color=blue>DO THIS:</font>** Let's generate __confusion matrices__ for the predicted results from the model with all features and the one with just the top 10.

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# Get confusion matrrix
cm_top10 = confusion_matrix(y_test, y_test_pred)

In [ ]:
# Plot the confusion matrix
cm_display = ConfusionMatrixDisplay(cm_top10).plot()

The confusion matrix on the left tell us that:
- Number of true negative ($tn$) = 292
- Number of false positive ($fp$) = 75
- Number of false negative ($fn$) = 17
- Number of true positive ($tp$) = 46

&#9989; <font color=red>**QUESTION:**</font> In __the next 1 min__, discuss with your neighbors: based on the above results? Do you feel that the model is doing well? Why and why not?

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

### ___7.3 Classification report___

&#9989; **<font color=orange>CAUTION:</font>** The metrics printed out are defined below. Unfortunately, we __do not have time__ to go through these. But for you to run an ML project, these, including ROC-AUC, are __foundational knowledge and you need to know the advantage and disadvantage of using them__. [Here is an excellent summary](https://machinelearningmastery.com/metrics-evaluate-machine-learning-algorithms-python/) of important metrics for evaluating ML models.

|Metric|Formula|
|---|---|
|Precision|$p = tp/(tp + fp)$|
|Recall|$r = tp / (tp + fn)$|
|F1 score|$f1 = 2(p\times r)/(p + r)$|
|Accuracy|$(tp+tn)/(tp+fp)$|
|Macro averge|$0.5\times score_{\text{class0}} + 0.5\times score_{\text{class1}}$|
|Weighted averge|$P_{\text{class0}}\times score_{\text{class0}} + P_{\text{class1}}\times score_{\text{class1}}$<br>$P$: proportion of a class.|

Support: number of each class (not a performance metric)


&#9989; **<font color=blue>DO THIS:</font>** Let's also generate a classification report to get a few other performance matrics.

In [ ]:
from sklearn.metrics import classification_report

# Set class names
targets = ["GM", "SM"]

report = classification_report(y_test, y_test_pred, target_names=targets)
print(report)

### &#9978; **<font color=purple>PAUSE: once you finish, please turn your attention to the instructor. </font>**

### ___7.4 Graphics that help with evaluation___

&#9989; **<font color=blue>DO THIS:</font>** Here we provide two examples: ROC-AUC curve and precision-recall curve. The red dotted line indicate how a naive classifer woul fair with __random guesses__. We only plot these curves for the model with top 10 features.

In [ ]:
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import PrecisionRecallDisplay

def plot_curves(X, y, estimator, background):

    fig, axs = plt.subplots(1, 2, figsize=(10,5))

    # ROC-AUC curve
    RocCurveDisplay.from_estimator(estimator, X, y, ax=axs[0])
    # Plot ROC-AUC background
    axs[0].plot([0, 1], [0, 1],'r--')

    # Precision-recall curve
    PrecisionRecallDisplay.from_estimator(estimator, X, y, ax=axs[1])
    axs[1].legend(loc='upper right')
    # Plot PR-curve background
    axs[1].plot([0, 1], [background, background],'r--')
    axs[1].set_ylim(-0.05,1.05)

    plt.show()

num_class0 = y_test[y_test==0].shape[0]
num_class1 = y_test[y_test==1].shape[0]
background = num_class1/(num_class0+num_class1)
print(background)

plot_curves(X_test_top10, y_test, rfc_loaded_top10, background)

&#9989; <font color=red>**QUESTION:**</font> The red lines are what we expect a random model will be performing. For the plot on the right-hand side, it is defined by the `background` value defined in the cell above. Why would it be cosndiered as `background` (i.e., random guess) for the right-hand plot?

&#9989; **<font color=blue>DO THIS:</font>** Let's also get these curves for training data.

In [ ]:
num_class0 = y_train[y_train==0].shape[0]
num_class1 = y_train[y_train==1].shape[0]
background = num_class1/(num_class0+num_class1)
print(background)

plot_curves(X_train_top10, y_train, rfc_loaded_top10, background)

&#9989; <font color=red>**QUESTION:**</font> In __the next 2 min__, discuss with your neighbor: why are the model performance for training data so much better than that for the testing set?

### &#9978; **<font color=purple>PAUSE: after your discussion, please turn your attention to the instructor. </font>**

---
<a name="step7"></a>
## __Step 8. Interpret model__

### ___8.1 Global interpetation using SHAP___

&#9989; **<font color=blue>DO THIS:</font>** Run the code below to get [SHAP (SHapley Additive exPlanations)](https://shap.readthedocs.io/en/latest/index.html) values that use game theory to determine how important each feature is in contributing to a prediction. There are MANY, MANY things you can do with SHAP and we will first figure out which features are more important than the others.

In [ ]:
import shap
shap.initjs()

# Get a TreeExplainer object using the model we have created
explainer = shap.TreeExplainer(rfc_loaded_top10)

# Use the TreeExplainer to get SHAP values for the training data
shap_values = explainer.shap_values(X_train_top10)

# Generate a summary plot
shap.summary_plot(shap_values, X_train_top10, sort=True)

We can see that `Fam_size` is the most important. Because we have only two classes, if a feature is important for classifyting one class, it must also be important for the other class.

### ___8.2 SHAP values of different instances___

&#9989; **<font color=blue>DO THIS:</font>** Another way to look at the SHAP values is by focusing on a particular class. In the example below, we focus on how different features contribute to the predictions of label=1 (SM):

In [ ]:
shap_for_label1 = shap_values[1]

# plot the shape value and color based on feature values
shap.summary_plot(shap_for_label1, X_train_top10, sort=True)

For each feature $x$, two SHAP values are generated for each instance: one for $x$'s contribution to class $0$ and the other for its contribution to class $1$. In the plot above, we are only looking at the contribution to class $1$ and each dot is an instance.

Look at Fam_size (family size), instances with higher features values (i.e., in larger families) also tend to have higher positive SHAP values (i.e., positive contribution to be in class 1).

&#9989; <font color=red>**QUESTION:**</font> In __the next 2 min__, discuss with your neighbor and interpret what the `Func_likelihood` feature's SHAP value distribution means where a higher feature values correlate with lower SHAP.

### &#9978; **<font color=purple>PAUSE: after your discussion, please turn your attention to the instructor. </font>**

-----